In [3]:
pip install --upgrade openai


  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/8d/36/47f191936fb61eb7efaee5e727fb962be82b0f66e979a32446d22f3f99cc/openai-1.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/14/fd/2f20c40b45e4fb4324834aea24bd4afdf1143390242c0b33774da0e2e34f/anyio-4.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/ed/76/9a17032880ed27f2dbd490c77a3431cbc80f47ba81534131de3c2846e736/pydantic-2.7.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install python-pptx

  Obtaining dependency information for python-pptx from https://files.pythonhosted.org/packages/72/49/6eee83072983473e9905ffddd5c2032b9a0ca4616425560d6d582287b467/python_pptx-0.6.23-py3-none-any.whl.metadata
  Obtaining dependency information for XlsxWriter>=0.5.7 from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/471.6 kB ? eta -:--:--
   -------- ------------------------------- 102.4/471.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------- ------- 378.9/471.6 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 471.6/471.6 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   ---------------------------------------- 159.9/159.9 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import openai

In [2]:
print(openai.__version__)

1.28.1


In [8]:
from flask import Flask, render_template, request, send_file
from pptx import Presentation
from pptx.util import Pt
import os
import io
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from openai import OpenAI

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/generate_ppt', methods=['POST'])
def generate_ppt():
    api_key = "sk-proj-TYcssHY0HC7h0uTnCDI4T3BlbkFJtZa3DvaZLAJctiACaze2"  # Fetch your API key from environment variables
    if not api_key:
        return {"error": "OpenAI API key not found."}

    print(api_key)
    client = OpenAI(api_key=api_key)

    TEMPLATE_FILE = 'C:\\Users\\adith\\Desktop\\New folder\\template3.pptx'
    if not os.path.exists(TEMPLATE_FILE):
        return {"error": "Template file not found."}

    data = request.get_json()
    if not data:
        return {"error": "No data received."}

    presentation_title = data.get('presentation_title')
    slide_contents = data.get('slide_contents')

    user_message_content = f"Create a presentation with 3 slides.\n" \
                           f"Presentation title: {presentation_title}\n" \
                           f"Slide content: {slide_contents}\n"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant that gives ideas for PowerPoint presentations."},
            {"role": "user", "content": user_message_content},
        ],
        model="gpt-3.5-turbo",
    )

    if chat_completion.choices[0].message.content:
        generated_text = chat_completion.choices[0].message.content

        presentation = Presentation(TEMPLATE_FILE)

        slides = generated_text.split("Slide")

        if len(slides) - 1 <= len(presentation.slides):
            for i, slide_content in enumerate(slides[1:], start=0):
                content_lines = slide_content.strip().split("\n")
                if len(content_lines) >= 2:
                    slide_title = content_lines[0].strip()
                    slide_content = content_lines[1:]  # Extract bullet points

                    slide = presentation.slides[i]
                    title = slide.shapes.title
                    content_box = slide.placeholders[1]

                    title.text = slide_title
                    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)

                    content_box.text = ""  # Clear existing text
                    content_frame = content_box.text_frame
                    content_frame.clear()  # Clear existing paragraphs
                    for point in slide_content:
                        p = content_frame.add_paragraph()
                        p.text = point.strip()
                        p.font.color.rgb = RGBColor(255, 255, 255)
                        p.space_after = Pt(14)  # Adjust spacing between bullet points

        else:
            return {"error": "The template has fewer slides than required."}

        pptx_bytes = io.BytesIO()
        presentation.save(pptx_bytes)
        pptx_bytes.seek(0)

        return send_file(
                pptx_bytes,
                mimetype='application/vnd.openxmlformats-officedocument.presentationml.presentation',
                as_attachment=True,
                download_name=f"{presentation_title}.pptx"
)

    else:
        return {"error": "No response from OpenAI."}


app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
